In [1]:
import numpy as np
def entropy(array):
    totalLength = len(array)  # Total number of values/entries
    pArr = split(array)       # Dictionary of unique value locations/indexes
    pLen = len(pArr)          # Number of unique values
    plist = list(pArr.keys()) # List of unique value number names.
    Hp = 0
    for k in range(0,pLen):
        nn = plist[k]
        vLen = len(pArr[nn])
        px = vLen / totalLength
        if px != 0:
            Hp += px*np.log2(1/px)
        
    return Hp  # return a scalar

In [8]:
import numpy as np
xd1 = np.array([0, 1, 0, 1, 0, 1])   # x data
yd1 = np.array([0, 1, 0, 1, 1, 1])   # y data
# Indicies =    0, 1, 2, 3, 4, 5   
x_split = split(xd1)
# ({0: array([0, 2, 4]), 1: array([1, 3, 5])}
#
xLen = len(xd1)       # Number of items in x data (Parent)

#parent entropy should be yd1
pe = entropy(xd1)     # Parent Entropy
pe1 = entropy(yd1)

# From the split of xd1; used as indexes into yd1; yields y_sub arrays: ys1, ys2
ys1 = [0,0,1]
ys2 = [1,1,1]
#
ce1 = entropy(ys1)    # Entropy of child 1
ce2 = entropy(ys2)    # Entropy of child 2

f3 = len(x_split[0])/xLen
f4 = len(x_split[1])/xLen
f1 = len(ys1)/xLen
f2 = len(ys2)/xLen
#
ig = pe1 - (ce1 * f4) - (ce2 * f3)
print("Info Gain = %f\nParent entropy = %f\nChild_1 entropy = %f\nChild_2 entropy = %f" %(ig,pe,ce1,ce2))
# Should be = ig = 0.4591
#-------------------------------------------------------------------------------------------

Info Gain = 0.459148
Parent entropy = 1.000000
Child_1 entropy = 0.918296
Child_2 entropy = 0.000000


In [1]:
%load_ext watermark
%watermark -d -u -a 'Kiet_Dinh' -v -p numpy,scipy,matplotlib,sklearn

Kiet_Dinh 
last updated: 2020-06-20 

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
scipy 1.4.1
matplotlib 3.1.3
sklearn 0.22.1


In [2]:
import numpy as np

In [1]:
def split(array):
    dictionary = {}
    a = np.unique(array)
#     print(a)
    for i in a:
        dictionary[i], = np.where(array == i)
    return dictionary # return the dictionary variable

NameError: name 'np' is not defined

In [4]:
split(np.array([0, 1, 2, 1, 0, 3, 1, 0, 1, 2]))

{0: array([0, 4, 7], dtype=int64),
 1: array([1, 3, 6, 8], dtype=int64),
 2: array([2, 9], dtype=int64),
 3: array([5], dtype=int64)}

In [5]:
def entropy(array):
    pdict = split(array)
    pi = []
    for i in pdict:
        pi.append(len(pdict[i])/array.size)
        
    entropy = 0
#     for i in pi:
#         entropy += -i*np.log2(i)
#     return entropy
    return -np.nansum(pi*np.log2(pi)) 

In [6]:
print(round(entropy(np.array([0, 1, 0, 1, 1, 0])), 4))
print(round(entropy(np.array([1, 2])), 4))
print(round(entropy(np.array([1, 1])), 4))
print(round(entropy(np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), 4))
print(round(entropy(np.array([0, 0, 0])), 4))
print(round(entropy(np.array([1, 1, 1, 0, 1, 4, 4, 2, 1])), 4))

1.0
1.0
-0.0
0.4395
-0.0
1.6577


In [7]:
def information_gain(x_array, y_array):
    
    parent_entropy = entropy(y_array)
    print(parent_entropy)
    split_dict = split(x_array)

    for key,value in split_dict.items():
        freq = value.size/x_array.size
        
        # ?
        child_entropy = entropy(y_array[value])
        # ?
        parent_entropy -= freq*child_entropy
        
    return parent_entropy


In [8]:
x = np.array([0, 1, 0, 1, 0, 1])
y = np.array([0, 1, 0, 1, 1, 1])
print(round(information_gain(x, y), 4))

x = np.array([0, 0, 1, 1, 2, 2])
y = np.array([0, 1, 0, 1, 1, 1])
print(round(information_gain(x, y), 4))

0.9182958340544896
0.4591
0.9182958340544896
0.2516


In [9]:
def make_tree(X, y):
    # Return array if node is empty or pure (1 example in leaf node)
    if y.shape[0] == 1 or y.shape[0] == 0:
        return y
    
    # Compute information gain for each feature
    gains = [information_gain(y,x_columns) for x_columns in X.T]

    # For np.max([len(X[i]) for i in range(len(X))])
#     for j in range(len(X[0])):
#         gains.append(round(information_gain(np.array([i[j] for i in X]),y), 4))
    
    # Early stopping if there is no information gain
    if all(i <= 1e-05 for i in gains):
        return y
    
    # Else, get best feature
    # argmax returns INDEX of the max value
    best_feature = np.argmax(gains)
    
    
    results = {}
    
    # Use the `split` function to split on the best feature
    subset_dict = split(X[:, best_feature])
    
    # Note that each entry in the dictionary returned by
    # split is an attribute_value:array_indices pair.
    # here, we are going to iterate over these key-value
    # pairs and select the respective examples for the
    # new child nodes
    
    for feature_value, train_example_indices in subset_dict.items():
        child_y_subset = y[train_example_indices]
        child_x_subset = X[train_example_indices]

        results["X_%d = %d" % (best_feature, feature_value)] = make_tree(child_x_subset, child_y_subset)
        
        
    return results

In [10]:
x1 = np.array([0, 0, 1, 1, 2, 2])
x2 = np.array([0, 1, 0, 1, 0, 1])
X = np.array([x1, x2]).T
y = np.array([0, 1, 0, 1, 1, 1])
print('Inputs:\n', X)
print('\nLabels:\n', y)
print('\nDecision tree:\n', make_tree(X, y))

Inputs:
 [[0 0]
 [0 1]
 [1 0]
 [1 1]
 [2 0]
 [2 1]]

Labels:
 [0 1 0 1 1 1]
1.584962500721156
1.0
1.584962500721156
-0.0
1.584962500721156
-0.0

Decision tree:
 {'X_1 = 0': {'X_0 = 0': array([0]), 'X_0 = 1': array([0]), 'X_0 = 2': array([1])}, 'X_1 = 1': array([1, 1, 1])}


In [11]:
class ID3DecisionTreeClassifer(object):
    def __init__(self):
        pass
    def fit(self, X, y):
        self.splits_ = make_tree(X,y)
        
    def _majority_vote(self, label_array):
        return np.argmax(np.bincount(label_array))
    
    def _traverse(self, x, d):
        if isinstance(d, np.ndarray):
            return d
        for key in d:
            name, value = key.split(' = ')
            feature_idx = int(name.split('_')[-1])
            value = int(value)
            if x[feature_idx] == value:
                return self._traverse(x, d[key])
            
    def predict(self, x):
        label_array = self._traverse(x,self.splits_)
        return self._majority_vote(label_array)

In [12]:
x1 = np.array([0, 0, 1, 1, 2, 2])
x2 = np.array([0, 1, 0, 1, 0, 1])
X = np.array([x1, x2]).T
y = np.array([0, 1, 0, 1, 1, 1])
# DO NOT EDIT OR DELETE THIS CELL
tree = ID3DecisionTreeClassifer()
tree.fit(X, y)
print(tree.predict(np.array([0, 0])))
print(tree.predict(np.array([0, 1])))
print(tree.predict(np.array([1, 0])))
print(tree.predict(np.array([1, 0])))
print(tree.predict(np.array([1, 1])))
print(tree.predict(np.array([2, 0])))
print(tree.predict(np.array([2, 1])))

1.584962500721156
1.0
1.584962500721156
-0.0
1.584962500721156
-0.0
0
1
0
0
1
1
1


In [13]:
from mlxtend.data import iris_data
X, y = iris_data()
print('Number of examples:', X.shape[0])
print('Number of features:', X.shape[1])
print('Unique class labels:', np.unique(y))

Number of examples: 150
Number of features: 4
Unique class labels: [0 1 2]


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 45, random_state=123, stratify=y)
print('Number of training examples:', X_train.shape[0])
print('Number of test examples:', X_test.shape[0])

Number of training examples: 105
Number of test examples: 45


In [15]:
def draw_bootstrap_sample(rng, X, y):
    sample_indices = np.arange(X_train.shape[0])
    bootstrap_indices = rng.choice(sample_indices,size=sample_indices.shape[0])
    return X[bootstrap_indices], y[bootstrap_indices]

In [16]:
# DO NOT EDIT OR DELETE THIS CELL
rng = np.random.RandomState(123)
X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
print('Number of training inputs from bootstrap round:', X_boot.shape[0])
print('Number of training labels from bootstrap round:', y_boot.shape[0])
print('Labels:\n', y_boot)

Number of training inputs from bootstrap round: 105
Number of training labels from bootstrap round: 105
Labels:
 [0 0 1 0 0 1 2 0 2 1 0 0 2 1 1 1 1 2 1 1 2 0 2 1 2 1 1 1 0 1 0 0 1 2 0 0 0
 0 2 1 1 2 1 2 1 1 2 1 2 0 1 1 2 2 1 0 1 0 2 2 0 1 0 2 0 0 0 0 1 2 0 0 1 0
 1 1 0 1 1 2 2 0 2 0 2 0 1 1 2 2 0 2 2 2 0 1 0 1 2 2 2 1 0 0 0]


In [17]:
from sklearn.tree import DecisionTreeClassifier
class BaggingClassifier(object):
    def __init__(self, num_trees=10, random_state=123):
        self.num_trees = num_trees
        self.rng = np.random.RandomState(random_state)
    def fit(self, X, y):
        self.trees_ = [DecisionTreeClassifier(random_state=self.rng) for i in range(self.num_trees)]
        for i in range(self.num_trees):
            X_boot, y_boot = draw_bootstrap_sample(self.rng, X, y)
            # YOUR CODE to
            # fit the trees in self.trees_ on the bootstrap samples
            self.trees_[i].fit(X_boot,y_boot)
    def predict(self, X):
        ary = np.zeros((X.shape[0], len(self.trees_)), dtype=np.int)
        for i in range(len(self.trees_)):
            ary[:, i] = self.trees_[i].predict(X)
            maj = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)),axis=1,arr=ary)
        return maj

In [18]:
model = BaggingClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('Individual Tree Accuracies:')
for tree in model.trees_:
    predictions = tree.predict(X_test)
    print('%.1f%%' % ((predictions == y_test).sum() / X_test.shape[0] * 100))
print('\nBagging Test Accuracy: %.1f%%' % ((predictions == y_test).sum() / X_test.shape[0] * 100))

Individual Tree Accuracies:
88.9%
93.3%
97.8%
93.3%
93.3%
93.3%
91.1%
97.8%
97.8%
97.8%

Bagging Test Accuracy: 97.8%


In [19]:
import numpy as np

In [20]:
rng = np.random.RandomState(123)
num_bootstrap = 200
all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype=np.int)
for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = DecisionTreeClassifier(random_state=66).fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred
main_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)),axis=0,arr=all_pred)

In [21]:
avg_bias = round(np.sum(main_predictions != y_test) / y_test.size,4)
avg_bias

0.0222

In [22]:
var = np.zeros(pred.shape)

# each bootstrap has a pred
# all_pred are 2d array of preds in all bootstraps

for pred in all_pred:
    var += (pred != main_predictions).astype(np.int)

var /= num_bootstrap
avg_var = round(var.sum()/y_test.shape[0],4)
avg_var

0.0346

In [23]:
rng = np.random.RandomState(123)
num_bootstrap = 200
all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype=np.int)
for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    model = BaggingClassifier(random_state=66)
    model.fit(X_boot, y_boot)
    pred= model.predict(X_test)
    all_pred[i] = pred
main_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)),axis=0,arr=all_pred)

In [24]:
avg_bias = round(np.sum(main_predictions != y_test) / y_test.size,4)
avg_bias

0.0222

In [25]:
var = np.zeros(pred.shape)

for pred in all_pred:
    var += (pred != main_predictions).astype(np.int)
var /= num_bootstrap

avg_var = round(var.sum()/y_test.shape[0],4)
avg_var

0.0272

In [26]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

rng = np.random.RandomState(123)
num_bootstrap = 200
all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype=np.int)
for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = AdaBoostClassifier(random_state=66).fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred
main_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)),axis=0,arr=all_pred)

In [27]:
avg_bias = round(np.sum(main_predictions != y_test) / y_test.size,4)
avg_bias


0.0444

In [28]:
var = np.zeros(pred.shape)

for pred in all_pred:
    # (np.array(something) == np.array(something)).astype(np.int), the condition is only for numpy array 
    var += (pred != main_predictions).astype(np.int)
var /= num_bootstrap

avg_var = round(var.sum()/y_test.shape[0],4)
avg_var

0.0328

##### Average Bias > 3.1, 0.0444 > 0.0222
##### Average variance slightly > 3.1, 0.0328 > 0.0346

In [29]:
from mlxtend.data import mnist_data
X, y = mnist_data()
print('Dimensions: %s x %s' % (X.shape[0], X.shape[1]))
print('1st row', X[0])

Dimensions: 5000 x 784
1st row [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.  51. 159. 253. 159.  50.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  48. 238. 252. 252. 252. 237.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  54.
 227. 253. 252. 239. 233. 252.  57.   6.   0. 

In [30]:
from mlxtend.preprocessing import shuffle_arrays_unison
X, y = shuffle_arrays_unison((X, y), random_seed=1)
X_train, y_train = X[:4500], y[:4500]
X_test, y_test = X[4500:], y[4500:]

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

model = RandomForestClassifier(n_estimators=100, random_state=123)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred)*100
print('Accuracy %.1f%%' % acc)

Accuracy 93.6%


In [32]:
from PIL import Image
import numpy as np

def load_image(file_name):
    img = Image.open(file_name)
    img.load()
    data = np.asarray(img, dtype=np.float)
    return data
x_image = load_image('5.png')

In [33]:
x_image.shape

(28, 28)

In [34]:
# The data needs to be represented as a vector (1 position for each feature)
x_transf = x_image.flatten()

In [35]:
x_transf.shape

(784,)

In [36]:
# Also, scikit-learn expects 2D arrays, so we need to add a dimension
x_transf = x_transf[..., np.newaxis].T
x_transf.shape

(1, 784)

In [37]:
print('Digit:', model.predict(x_transf)[0])

Digit: 5
